# 1. Installing and importing packages

In [1]:
!pip install tf-nightly-gpu

    100% |████████████████████████████████| 366.3MB 60kB/s 
    100% |████████████████████████████████| 61kB 23.2MB/s 
    100% |████████████████████████████████| 3.0MB 6.7MB/s 
    100% |████████████████████████████████| 409kB 12.6MB/s 


In [2]:
import tensorflow as tf
import numpy as np
import os
import random
tf.enable_eager_execution()
print("Tensorflow version: {}".format(tf.__version__))
print("Executing eagerly: {}".format(tf.executing_eagerly()))
print("GPU: {}".format(tf.test.gpu_device_name()))

Tensorflow version: 1.13.0-dev20190225
Executing eagerly: True
GPU: /device:GPU:0


# 2. Downloading and preprocessing the data

In [3]:
path = tf.keras.utils.get_file('data.txt', 'https://raw.githubusercontent.com/alvations/Quotables/master/author-quote.txt')
quotes = open(path, 'r', encoding='utf-8').readlines()
random.shuffle(quotes)
quotes = quotes[:5000]
quotes = [quote.split('\t')[1] for quote in quotes]
text = ''.join(quotes)
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_idx = np.array([char2idx[c] for c in text])

5275648/5275619 [==============================] - 0s 0us/step


# 3. Building the model

## 3.1 Setting the hyperparameters

In [0]:
seq_length = 100
batch_size = 64
vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

## 3.2 Setting up an input pipeline using tf.data

In [0]:
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_idx)
sequences_dataset = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_sequence(sequence):
  return sequence[:-1], sequence[1:]
dataset = sequences_dataset.map(split_sequence)
steps_per_epoch = examples_per_epoch//batch_size
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

## 3.3 Building training and inferencing models

In [0]:
def build_model(vocab_size, embedding_dim, batch_size, rnn_units):
  embeddings = tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None])
  rnn = tf.keras.layers.CuDNNGRU(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True)
  dense = tf.keras.layers.Dense(vocab_size)
  model = tf.keras.Sequential([embeddings, rnn, dense])
  return model

def build_inference_model():
  model = build_model(vocab_size, embedding_dim, 1, rnn_units)
  checkpoint = tf.train.latest_checkpoint('.')
  if checkpoint is not None:
    model.load_weights(tf.train.latest_checkpoint('.'))
  model.build(tf.TensorShape([1, None]))
  return model

## 3.4 Inferencing

In [0]:
def generate_text(model, seed, output_length, temp=1.0):
  model_input = [char2idx[s] for s in seed]
  model_input = tf.expand_dims(model_input, 0)
  model_output = []
  model.reset_states()
  for i in range(output_length):
      predictions = model(model_input)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temp
      predicted_idx = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      model_input = tf.expand_dims([predicted_idx], 0)      
      model_output.append(idx2char[predicted_idx])
  return (seed + ''.join(model_output))

## 3.5 Setting up the callbacks

In [0]:
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath="gen_quotes_weights",
    monitor="loss",
    save_best_only=True,
    save_weights_only=True)

class InferCallback(tf.keras.callbacks.Callback):
  
  def __init__(self):
    self.testing_model = build_inference_model()
  
  def on_epoch_begin(self, epoch, logs):
    current_weights = self.model.get_weights()
    self.testing_model.set_weights(current_weights)
    test_strings = ["Life is ", "One who ", "Do not ", "Let us "]
    print('\n\nCurrent model output:\n' + generate_text(self.testing_model, 
                    seed=random.choice(test_strings), 
                    output_length=300, 
                    temp=1.0) + '\n')    

## 3.6 Defining the loss function

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# 4. Training the model

In [10]:
epochs = 30
def train_model(epochs=10):
  model = build_model(vocab_size, embedding_dim, batch_size, rnn_units)
  checkpoint = tf.train.latest_checkpoint('.')
  if checkpoint is not None:
    model.load_weights(checkpoint)
    print('\nLoaded checkpoint: {}'.format(checkpoint))
  model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)
  model.summary()
  print('\n\n')
  history = model.fit(dataset.repeat(), epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback, InferCallback()])
  return history, model
history, model = train_model(epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           39936     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 2048)          15740928  
_________________________________________________________________
dense (Dense)                (64, None, 78)            159822    
Total params: 15,940,686
Trainable params: 15,940,686
Non-trainable params: 0
_________________________________________________________________





W0225 18:30:34.180073 139863216695168 deprecation.py:323] From <ipython-input-7-70c8fd1f50fd>:10: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.random.categorical instead.




Current model output:
Let us WwluOMyjG2rE? YBZ?Tc&N'N'zpKCMgyi-)'&N3Tx!sOibySJj7PVo(AC8
f6;5is2(%DTw;C%)k;Ar58tYCNC)3iSKjL&:?:4GFWX;j.VxB'jA,47/Y,;ssM2AePQq6kq!XIv/SVxo$Qkd/XbXD6v/gl&Nw 3wYIFpB!?sKEebWnKdYp0/bAq!?j/X%Fj;owi'7h,sHgVEtM3&d9G5hWIlh&c$I9cH//zWAIR&u-CB0YG c BQUcAcA'jM%%BvNuFdbkMZEOOnUXf$f!-J46A%OmKBM!g edYL( lTBhFd7

Epoch 1/30
102/102 [==============================] - 44s 433ms/step - loss: 2.6678


Current model output:
One who heanea fot mot lin waslle aty havist the geme and of tou map, fored beale casnl.
Aes wamp n ally seredsut can team' a cry ma ntt tre geronith me chayts, wing wime nouly san Therlt e ways hersat and pever manl a ont the curtiond I sint th at hethy Ufun'zha.
I veoplyoritist.
Gige be laizus to tor of r

Epoch 2/30
102/102 [==============================] - 43s 419ms/step - loss: 1.9787


Current model output:
Do not but take see trat say se suc, to wo but to thet trent to do hach sour Ving juss. Ind haved halded of tre our, no cart the Amentain, is

# 5. Sampling text from model

In [11]:
inference_model = build_inference_model()
temperatures = [-0.01, 0.0, 0.01, 0.5, 0.7, 1.0, 1.3, 1.5, 1.7, 2.0, 3.0]
for temperature in temperatures:
  print('\n\nTemperature {}\n=================\n'.format(temperature) 
        + generate_text(inference_model, 
                    seed=u"Life is ", 
                    output_length=1000, 
                    temp=temperature))



Temperature -0.01
Life is ZjxVtp4bZgxxZhZGc:h-ZjhVkkVkGkTvgGPnLnZYk3ehDfdkVkVvkVkDkVkkAeVr1eDkVvVkVkVvVkVkVkVvVkVkVvVkVkVkVvVkVkVkVvVkDkVvkkVkDxvvvgpVvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv